<a href="https://colab.research.google.com/github/PalmerTurley34/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/LS_DS_434_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


### Data Cleaning

In [55]:
import pandas as pd
PATH = '/content/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv'

df = pd.read_csv(PATH)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
print(df.shape)
df.nunique()

(7043, 21)


customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64

In [9]:
df['PaymentMethod'].value_counts()

Electronic check             2365
Mailed check                 1612
Bank transfer (automatic)    1544
Credit card (automatic)      1522
Name: PaymentMethod, dtype: int64

In [56]:
binary_columns = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']

df[binary_columns] = df[binary_columns].replace('Yes', 1)
df[binary_columns] = df[binary_columns].replace('No', 0)

df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,1,0,1,0,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,0,0,34,1,No,DSL,Yes,No,Yes,No,No,No,One year,0,Mailed check,56.95,1889.5,0
2,3668-QPYBK,Male,0,0,0,2,1,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,Male,0,0,0,45,0,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,Female,0,0,0,2,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,70.70,151.65,1


In [57]:
df.describe()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,Churn
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,0.483033,0.299588,32.371149,0.903166,0.592219,64.761692,0.265370
std,0.368612,0.499748,0.458110,24.559481,0.295752,0.491457,30.090047,0.441561
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,0.000000
25%,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,35.500000,0.000000
50%,0.000000,0.000000,0.000000,29.000000,1.000000,1.000000,70.350000,0.000000
75%,0.000000,1.000000,1.000000,55.000000,1.000000,1.000000,89.850000,1.000000
max,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,118.750000,1.000000


In [35]:
!pip install category_encoders

     |████████████████████████████████| 81kB 2.2MB/s 


In [41]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)

In [58]:
cat_cols = ['gender', 'MultipleLines', 'InternetService', 'OnlineSecurity',
            'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
            'StreamingMovies', 'Contract', 'PaymentMethod']

df = encoder.fit_transform(df)

In [59]:
df.head()

,customerID,gender_Female,gender_Male,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines_No phone service,MultipleLines_No,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_Yes,OnlineSecurity_No internet service,OnlineBackup_Yes,OnlineBackup_No,OnlineBackup_No internet service,DeviceProtection_No,DeviceProtection_Yes,DeviceProtection_No internet service,TechSupport_No,TechSupport_Yes,TechSupport_No internet service,StreamingTV_No,StreamingTV_Yes,StreamingTV_No internet service,StreamingMovies_No,StreamingMovies_Yes,StreamingMovies_No internet service,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling,PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,29.85,29.85,0
1,5575-GNVDE,0,1,0,0,0,34,1,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,56.95,1889.5,0
2,3668-QPYBK,0,1,0,0,0,2,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,53.85,108.15,1
3,7795-CFOCW,0,1,0,0,0,45,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,42.30,1840.75,0
4,9237-HQITU,1,0,0,0,0,2,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,70.70,151.65,1


### Baseline Model

In [67]:
X = df.drop(columns=['customerID', 'MonthlyCharges', 'TotalCharges'])
y = df['MonthlyCharges']

In [68]:
X.head()

,gender_Female,gender_Male,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines_No phone service,MultipleLines_No,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_Yes,OnlineSecurity_No internet service,OnlineBackup_Yes,OnlineBackup_No,OnlineBackup_No internet service,DeviceProtection_No,DeviceProtection_Yes,DeviceProtection_No internet service,TechSupport_No,TechSupport_Yes,TechSupport_No internet service,StreamingTV_No,StreamingTV_Yes,StreamingTV_No internet service,StreamingMovies_No,StreamingMovies_Yes,StreamingMovies_No internet service,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling,PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),Churn
0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0
1,0,1,0,0,0,34,1,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0
2,0,1,0,0,0,2,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1
3,0,1,0,0,0,45,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0
4,1,0,0,0,0,2,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,1


In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, Adamax

adam = Adam()

model = Sequential([
                    Dense(52, activation='sigmoid', input_dim=40),
                    Dense(53, activation='sigmoid'),
                    Dense(51, activation='sigmoid'),
                    Dense(1)
])

In [71]:
model.compile(optimizer=adam, loss='mse', metrics=['mse', 'mae'])

In [72]:
model.fit(X, y, epochs=25)

Epoch 1/25
221/221 [==============================] - 0s 1ms/step - loss: 4262.4609 - mse: 4262.4609 - mae: 57.8333
Epoch 2/25
221/221 [==============================] - 0s 2ms/step - loss: 3326.9724 - mse: 3326.9724 - mae: 49.1500
Epoch 3/25
221/221 [==============================] - 0s 1ms/step - loss: 2645.1294 - mse: 2645.1294 - mae: 42.7687
Epoch 4/25
221/221 [==============================] - 0s 1ms/step - loss: 2133.4556 - mse: 2133.4556 - mae: 38.8866
Epoch 5/25
221/221 [==============================] - 0s 2ms/step - loss: 1744.5199 - mse: 1744.5199 - mae: 35.7393
Epoch 6/25
221/221 [==============================] - 0s 2ms/step - loss: 1444.9648 - mse: 1444.9648 - mae: 32.9666
Epoch 7/25
221/221 [==============================] - 0s 2ms/step - loss: 1240.2977 - mse: 1240.2977 - mae: 30.7940
Epoch 8/25
221/221 [==============================] - 0s 2ms/step - loss: 1104.0692 - mse: 1104.0692 - mae: 29.2355
Epoch 9/25
221/221 [==============================] - 0s 2ms/step - loss

### Hyperparameter Tuning

In [73]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [107]:
def create_model():
  adam = Adam()
  adamax = Adamax()

  model = Sequential()
  model.add(Dense(52, activation='sigmoid', input_dim=40))
  model.add(Dense(53, activation='sigmoid'))
  model.add(Dense(51, activation='sigmoid'))
  model.add(Dense(1))
  model.compile(optimizer=adam, loss='mse', metrics=['mse'])
  return model

In [108]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [109]:
param_grid = {'batch_size': [16, 32, 64],
              'epochs': [10, 25, 50]}

In [110]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, scoring='neg_mean_squared_error')
grid_result = grid.fit(X,y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: -3049.2082278135895 using {'batch_size': 16, 'epochs': 10}


In [111]:
# learning rate
def create_model(lr):
  adam = Adam(learning_rate=lr)
  adamax = Adamax()

  model = Sequential()
  model.add(Dense(52, activation='sigmoid', input_dim=40))
  model.add(Dense(53, activation='sigmoid'))
  model.add(Dense(51, activation='sigmoid'))
  model.add(Dense(1))
  model.compile(optimizer=adam, loss='mse', metrics=['mse'])
  return model

In [112]:
model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [16],
              'epochs': [10],
              'lr': [.001, .01, .02]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, scoring='neg_mean_squared_error')
grid_result = grid.fit(X,y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: -3049.2082278135895 using {'batch_size': 16, 'epochs': 10, 'lr': 0.001}


In [113]:
# optimizer
def create_model(lr, opt):
  optimizer = opt(learning_rate=lr)


  model = Sequential()
  model.add(Dense(52, activation='sigmoid', input_dim=40))
  model.add(Dense(53, activation='sigmoid'))
  model.add(Dense(51, activation='sigmoid'))
  model.add(Dense(1))
  model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
  return model

In [114]:
model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [16],
              'epochs': [10],
              'lr': [.001],
              'opt': [Adam, Adamax]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, scoring='neg_mean_squared_error')
grid_result = grid.fit(X,y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: -3049.2082278135895 using {'batch_size': 16, 'epochs': 10, 'lr': 0.001, 'opt': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>}


In [115]:
# activation functions
def create_model(lr, opt, act):
  optimizer = opt(learning_rate=lr)


  model = Sequential()
  model.add(Dense(52, activation=act, input_dim=40))
  model.add(Dense(53, activation=act))
  model.add(Dense(51, activation=act))
  model.add(Dense(1))
  model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
  return model

In [116]:
model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [16],
              'epochs': [10],
              'lr': [.001],
              'opt': [Adam],
              'act': ['relu', 'sigmoid']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, scoring='neg_mean_squared_error')
grid_result = grid.fit(X,y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: -3049.2082278135895 using {'act': 'relu', 'batch_size': 16, 'epochs': 10, 'lr': 0.001, 'opt': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>}


In [117]:
# weight initialization
def create_model(lr, opt, act, weight_int):
  optimizer = opt(learning_rate=lr)


  model = Sequential()
  model.add(Dense(52, activation=act, input_dim=40, kernel_initializer=weight_int))
  model.add(Dense(53, activation=act))
  model.add(Dense(51, activation=act))
  model.add(Dense(1))
  model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
  return model

In [118]:
model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [16],
              'epochs': [10],
              'lr': [.001],
              'opt': [Adam],
              'act': ['relu'],
              'weight_int': ['glorot_uniform', 'uniform']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, scoring='neg_mean_squared_error')
grid_result = grid.fit(X,y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: -3049.2082278135895 using {'act': 'relu', 'batch_size': 16, 'epochs': 10, 'lr': 0.001, 'opt': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'weight_int': 'glorot_uniform'}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?